# Tutorial

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from multispline.spline import BicubicSpline

In [10]:
X = np.linspace(0, 2*np.pi, 80)
Y = np.linspace(0, 9*np.pi, 80)
Z = np.sin(X[:, None]) * np.cos(Y[None, :])

In [11]:
bspl = BicubicSpline(X, Y, Z)

In [12]:
bspl(0.1, 0.1) - np.sin(0.1) * np.cos(0.1)

np.float64(3.984825873701303e-06)